In [1]:
!pip install records boto3 psycopg2-binary &> /dev/null

In [2]:
import boto3
import records

In [3]:
db_url = boto3.client('ssm').get_parameter(Name='/bikefinder/dev/db_url', WithDecryption=True)['Parameter']['Value']
db = records.Database(db_url)

In [4]:
# number of bikes in operaton per day per provider
db.query(
    """
    ( -- number of unique bike_ids per provider (except ofo)
      select created::date, provider, count(distinct bike_id) num_bikes
      from bike_locations
      where provider<>'ofo'
      group by provider, created::date
    )
    union
    ( -- max number of ofo bikes available through out the day
      with ofo as (select * from bike_locations where provider='ofo'),
        counts_per_batch as (select count(bike_id), bike_id, created::date from ofo
                             group by bike_id, created::date)
      select created, 'ofo' provider, max("count") num_bikes
      from counts_per_batch
      group by created
    )
    order by created, provider;
    """).export('df')

,created,provider,num_bikes
0,2018-02-18,JUMP,54
1,2018-02-18,limebike,369
2,2018-02-18,mobike,464
3,2018-02-18,ofo,338
4,2018-02-18,spin,163


In [5]:
# min,avg,max lifetime of bikes per provider (except ofo, of course)
db.query(
    """
    with bike_lifetimes as (
        select bike_id, provider, max(created) - min(created) age
        from bike_locations
        where provider<>'ofo'
        group by bike_id, provider
    )
    select provider, min(age), avg(age), max(age)
    from bike_lifetimes
    group by provider;
    """).export('df')

,provider,min,avg,max
0,JUMP,00:00:00,02:28:51.870150,02:59:57.963220
1,mobike,00:00:04.960375,02:55:09.998301,03:00:12.491747
2,spin,00:10:00.022413,02:53:27.413846,02:59:58.682459
3,limebike,00:00:00,02:56:33.432677,02:59:59.003446


In [6]:
db.close()
db._engine.dispose()